In [1]:
import tensorflow as tf
import numpy as np

def readTfRecordSamples(tfrecords_filename):
    tf_record_iter = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    for string_record in tf_record_iter:
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
        print(example)  # this prints the abovementioned 4 keys but NOT audio_embeddings

        # the first label can be then parsed like this:
        label = (example.features.feature['labels'].int64_list.value[0])
        print('label 1: ' + str(label))

        audio_embedding = (example.features.feature['audio_embedding'].bytes_list.value[0])

In [3]:
audio_record = '/audioset_features_embeddings/eval/_1.tfrecord'
vid_ids = []
labels = []
start_time_seconds = []
end_time_seconds = []
feat_audio = []
count = 0
for example in tf.io.tf_record_iterator(audio_record):
    tf_example = tf.train.Example.FromString(example)

    vid_ids.append(tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding='UTF-8'))
    labels.append(tf_example.features.feature['labels'].int64_list.value)
    start_time_seconds.append(tf_example.features.feature['start_time_seconds'].float_list.value)
    end_time_seconds.append(tf_example.features.feature['end_time_seconds'].float_list.value)

    tf_seq_example = tf.train.SequenceExample.FromString(example)
    n_frames = len(tf_seq_example.feature_lists.feature_list['audio_embedding'].feature)

    sess = tf.InteractiveSession()
    rgb_frame = []
    audio_frame = []
    # iterate through frames
    for i in range(n_frames):
        audio_frame.append(tf.cast(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['audio_embedding'].feature[i].bytes_list.value[0],tf.uint8)
                       ,tf.float32).eval())

    sess.close()
    feat_audio.append([])

    feat_audio[count].append(audio_frame)
    count+=1

AttributeError: module 'tensorflow._api.v2.io' has no attribute 'tf_record_iterator'

In [ ]:
readTfRecordSamples(audio_record)

In [9]:
# import tensorflow.compat.v1 as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

feature_names = 'audio_embedding'

def prepare_serialized_examples(serialized_example, max_quantized_value=2, min_quantized_value=-2):

    contexts, features = tf.parse_single_sequence_example(
        serialized_example,
        context_features={"video_id": tf.FixedLenFeature([], tf.string),
                          "labels": tf.VarLenFeature(tf.int64)},
        sequence_features={'audio_embedding' : tf.FixedLenSequenceFeature([10], dtype=tf.string)
    })

    decoded_features = tf.reshape(
    tf.cast(tf.decode_raw(features['audio_embedding'], tf.uint8), tf.float32),
    [-1, 128])

    return contexts, decoded_features


filename = '/audioset_v1_embeddings/eval_train/_1.tfrecord'
filename_queue = tf.train.string_input_producer([filename], num_epochs=1)

reader = tf.TFRecordReader()

with tf.Session() as sess:
    
    _, serialized_example = reader.read(filename_queue)
    context, features = prepare_serialized_examples(serialized_example)

    init_op = tf.initialize_all_variables()
    sess.run(init_op)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    print(sess.run(features))

    coord.request_stop()
    coord.join(threads)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.NotFoundError'>, Container localhost does not exist. (Could not find resource: localhost/input_producer/limit_epochs/epochs)
	 [[{{node input_producer/limit_epochs/ResourceCountUpTo}}]]


I0000 00:00:1730339253.528518 15267134 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2024-10-30 21:47:33.534575: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at count_up_to_op.cc:71 : NOT_FOUND: Container localhost does not exist. (Could not find resource: localhost/input_producer/limit_epochs/epochs)
2024-10-30 21:47:33.534591: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: NOT_FOUND: Container localhost does not exist. (Could not find resource: localhost/input_producer/limit_epochs/epochs)
	 [[{{node input_producer/limit_epochs/ResourceCountUpTo}}]]
2024-10-30 21:47:33.537394: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at count_up_to_op.cc:71 : NOT_FOUND: Resource localhost/input_producer_1/limit_epochs/epochs/N10tensorflow3VarE does not exist.
2024-10-30 21:47:33.543590: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with stat

OutOfRangeError: Graph execution error:

Detected at node 'ReaderReadV2_1' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/Users/amblu/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    File "/Users/amblu/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    File "/Users/amblu/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start
    File "/Users/amblu/Library/Python/3.11/lib/python/site-packages/tornado/platform/asyncio.py", line 205, in start
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once
    File "/Library/Frameworks/Python.framework/Versions

In [ ]:
import tensorflow as tf

feature_names = 'audio_embedding'

def prepare_serialized_examples(serialized_example, max_quantized_value=2, min_quantized_value=-2):

    contexts, features = tf.parse_single_sequence_example(
        serialized_example,
        context_features={"video_id": tf.FixedLenFeature([], tf.string),
                          "labels": tf.VarLenFeature(tf.int64)},
        sequence_features={'audio_embedding' : tf.FixedLenSequenceFeature([10], dtype=tf.string)
    })

    decoded_features = tf.reshape(
    tf.cast(tf.decode_raw(features['audio_embedding'], tf.uint8), tf.float32),
    [-1, 128])

    return contexts, decoded_features


filename = '/audioset_v1_embeddings/eval_train/_1.tfrecord'
filename_queue = tf.data.Dataset.from_tensor_slices(filename)

reader = tf.TFRecordReader()

with tf.Session() as sess:
    
    _, serialized_example = reader.read(filename_queue)
    context, features = prepare_serialized_examples_(serialized_example)

    init_op = tf.initialize_all_variables()
    sess.run(init_op)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    print(sess.run(features))

    coord.request_stop()
    coord.join(threads)

In [ ]:
import tensorflow as tf

feature_names = 'audio_embedding'

def prepare_serialized_examples(serialized_example, max_quantized_value=2, min_quantized_value=-2):
    # Define the feature description for parsing
    feature_description = {
        "video_id": tf.io.FixedLenFeature([], tf.string),
        "labels": tf.io.VarLenFeature(tf.int64),
        "audio_embedding": tf.io.FixedLenFeature([10], dtype=tf.string),
    }

    # Parse the serialized example
    contexts, features = tf.io.parse_single_sequence_example(
        serialized_example,
        context_features={"video_id": feature_description["video_id"],
                         "labels": feature_description["labels"]},
        sequence_features={"audio_embedding": feature_description["audio_embedding"]}
    )

    # Decode the audio embedding
    decoded_features = tf.reshape(
        tf.cast(tf.io.decode_raw(features['audio_embedding'], tf.uint8), tf.float32),
        [-1, 128]
    )

    return contexts, decoded_features

# Filepath for the TFRecord file
filename = '/audioset_v1_embeddings/bal_train/2a.tfrecord'

# Create a dataset from the TFRecord file
raw_dataset = tf.data.TFRecordDataset(filename)

# Function to map the parsing function to the dataset
def map_func(serialized_example):
    return prepare_serialized_examples(serialized_example)

# Map the parsing function to the dataset
parsed_dataset = raw_dataset.map(map_func)

# Iterate through the dataset
for contexts, features in parsed_dataset:
    print(features)